# RDMS API – Retrieve Point Connect Data
### Retrieve data of multiple curves, merge them together using value_date and convert to pandas dataframe
-----------------------------------------------------------------------------------------
*This source code is provided AS IS with no warranty or guarantee of fit for purpose.*

*Copyright Refinitiv 2023.             All rights reserved.*

-----------------------------------------------------------------------------------------

First, declare variables required for API calls

- Replace '## YOUR API KEY ##' with your RDMS API Key
- Declare the 'curve_id_list' with curve IDs that you want to retrieve the data
- Replace 'your-rdms-instance' with your RDMS instance

In [11]:
import requests
import pandas as pd

api_key = '## YOUR API KEY ##'
curve_id_list = ['103425139','103425132','103425133','103425135','103425137','103425140','103425142','103425144','103425145']

base_url = 'https://your-rdms-instance.rdms.refinitiv.com/api'

payload={}
headers = {
  'accept': 'text/plain',
  'Authorization': api_key
}

In [12]:
def get_metadata(curve_id, tag):
    """ This function call the metadata endpoint and get specific metadata tag of the curve_id"""
    
    url = base_url + '/v1/Metadata/' + curve_id
    response = requests.request('GET', url, headers=headers, data=payload)
    curve_metadata = response.json()

    tag_name = [metadata_tag['name'] for metadata_tag in curve_metadata['tags']]
    tag_index = tag_name.index(tag)

    tag_value = curve_metadata['tags'][tag_index]['value']
    return tag_value

In [13]:
# Here's an example of the usage of function get_metadata
# Print out curve ID, Curve Name and Geography.is (from the response of metadata endpoint)
for curve_id in curve_id_list:
    print(curve_id + ": " + get_metadata(curve_id,'Curve Name') + get_metadata(curve_id,'Geography.is'))

103425139: AgWea.SYN_TMP_AVG_L1.USA;;USA.IA.1950;PC;R.CFSR;Syn;;D.1;T;°C;AVGF;D1950 Central (IA)
103425132: AgWea.SYN_TMP_AVG_L1.USA;;USA.IA.1910;PC;R.CFSR;Syn;;D.1;T;°C;AVGF;D1910 Northwest (IA)
103425133: AgWea.SYN_TMP_AVG_L1.USA;;USA.IA.1920;PC;R.CFSR;Syn;;D.1;T;°C;AVGF;D1920 North Central (IA)
103425135: AgWea.SYN_TMP_AVG_L1.USA;;USA.IA.1930;PC;R.CFSR;Syn;;D.1;T;°C;AVGF;D1930 Northeast (IA)
103425137: AgWea.SYN_TMP_AVG_L1.USA;;USA.IA.1940;PC;R.CFSR;Syn;;D.1;T;°C;AVGF;D1940 West Central (IA)
103425140: AgWea.SYN_TMP_AVG_L1.USA;;USA.IA.1960;PC;R.CFSR;Syn;;D.1;T;°C;AVGF;D1960 East Central (IA)
103425142: AgWea.SYN_TMP_AVG_L1.USA;;USA.IA.1970;PC;R.CFSR;Syn;;D.1;T;°C;AVGF;D1970 Southwest (IA)
103425144: AgWea.SYN_TMP_AVG_L1.USA;;USA.IA.1980;PC;R.CFSR;Syn;;D.1;T;°C;AVGF;D1980 South Central (IA)
103425145: AgWea.SYN_TMP_AVG_L1.USA;;USA.IA.1990;PC;R.CFSR;Syn;;D.1;T;°C;AVGF;D1990 Southeast (IA)


In [14]:
# set the min and max value date of the data
min_value_date = '2023-05-20T00:00:00'
max_value_date = '2023-05-30T00:00:00'

# create empty dataframe to store the result
output_df = pd.DataFrame()

for curve_id in curve_id_list:
    # for each curve ID in curve ID list
    # get geography metadata of curve ID and stored into variable curve_geography, to be used as a column name
    curve_geography = get_metadata(curve_id,'Geography.is')
    
    # Get curve value data
    # set the url with parameters (more parameters can be added, check available parameters in the RDMS API Swagger)
    url = base_url + f'/v1/CurveValues/{curve_id}?MinValueDate={min_value_date}&MaxValueDate={max_value_date}'
    response = requests.request('GET', url, headers=headers, data=payload)
    
    # Convert response from JSON format to pandas dataframe
    df = pd.json_normalize(response.json())
    
    # Drop unused columns and rename the columns
    df.drop(['scenarioID', 'forecastDate'], axis=1, inplace=True)
    df.rename(columns={'valueDate': 'date', 'value': curve_geography}, inplace=True)
    #display(df)
    
    # Put the df into output_df, merge dataframes together using date column
    if output_df.empty:
        output_df = output_df.append(df, ignore_index=True)
    else:
        output_df = output_df.merge(df, how='inner', on='date')

In [15]:
# Display final output dataframe
display(output_df)

,date,D1950 Central (IA),D1910 Northwest (IA),D1920 North Central (IA),D1930 Northeast (IA),D1940 West Central (IA),D1960 East Central (IA),D1970 Southwest (IA),D1980 South Central (IA),D1990 Southeast (IA)
0,2023-05-20T00:00:00+00:00,14.2,12.9,14.0,13.1,14.4,13.6,14.7,13.6,13.5
1,2023-05-21T00:00:00+00:00,16.5,17.8,17.1,15.8,18.0,16.0,17.0,15.8,15.7
2,2023-05-22T00:00:00+00:00,18.6,19.8,18.8,17.2,19.4,17.9,18.4,18.2,18.6
3,2023-05-23T00:00:00+00:00,19.7,21.0,20.1,18.1,20.9,18.8,20.5,19.0,18.4
4,2023-05-24T00:00:00+00:00,20.5,21.5,20.9,18.0,21.6,20.3,21.0,19.3,21.0
5,2023-05-25T00:00:00+00:00,19.2,21.4,18.0,14.7,21.9,16.1,21.7,20.3,19.1
6,2023-05-26T00:00:00+00:00,16.8,19.7,16.7,13.4,20.0,14.4,19.7,16.8,15.4
7,2023-05-27T00:00:00+00:00,17.1,18.6,17.5,15.7,18.4,16.5,17.6,15.9,17.3
8,2023-05-28T00:00:00+00:00,18.9,19.9,19.4,17.6,20.0,18.8,19.8,17.3,18.5
9,2023-05-29T00:00:00+00:00,20.3,22.1,20.9,20.4,22.5,20.4,21.8,20.4,19.2


## Related Resource
- [RDMS Web Application](https://agsprod1.rdms.refinitiv.com/)
    - [Generate API key](https://agsprod1.rdms.refinitiv.com/Identity/Account/Manage/ApiKeys)
- [RDMS swagger](https://agsprod1.rdms.refinitiv.com/api/swagger/index.html)
- [Refinitiv Devportal](https://developers.refinitiv.com/en)
    - [Refinitiv Data Management Solution (RDMS)](https://developers.refinitiv.com/en/api-catalog/rdms/rdms)
- [Q&A forum](https://community.developers.refinitiv.com/index.html)
    - [RDMS tag](https://community.developers.refinitiv.com/tags/56131/rdms.html)